In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor


In [2]:
def record_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the first table
    table = soup.find("table")
    
    # Extract rows
    rows = []
    ds = []
    for tr in table.find_all("tr"):
        cells = [td.text.strip() for td in tr.find_all("td")]
        if cells:
            rows.append(cells)

        ds.append(rows)

    scrapped_ds = [dict(rec) for rec in ds]
    return scrapped_ds

def get_rec(year):
    base_url = f"https://www.planecrashinfo.com/{year}"
    # Fetch the HTML content
    url = f"{base_url}/{year}"
    response = requests.get(f"{url}.htm")
    # response = requests.get('https://example.com')
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the first table
    table = soup.find("table")

    if table is None:
        return None

    # Extract headers
    headers = [td.text.strip() for td in table.find_all("tr")[0]]

    # Extract rows
    rows = []
    cnt = 0
    for tr in table.find_all("tr")[1:]:  # Skip header row
        cells = [td.text.strip() for td in tr.find_all("td")]
        if cells:
            rows.append(cells)
            cnt += 1

    # # Convert to DataFrame
    # df = pd.DataFrame(rows, columns=headers)

    # # Display DataFrame
    # df.head()
    
    urls = []
    for i in range(1, cnt+1):
        urls.append(f"{url}-{i}.htm")
    
    with ThreadPoolExecutor() as executor:
        results = executor.map(record_page, urls)
    
    print(results)
    return results



In [ ]:
years = range(1986,1990)

for yr in years:
    res = get_rec(yr)

    if res is None:
        print(f'failed: {yr}')
    else:
        print(yr)
    
    

<generator object Executor.map.<locals>.result_iterator at 0x1159d9b70>
1986
<generator object Executor.map.<locals>.result_iterator at 0x114fbab60>
1987


136.71s - thread._ident is None in _get_related_thread!
